In [ ]:
"""
Comprehensive Unit Tests for DSP Core Module

This test suite validates the correctness of hand-written FFT, STFT, and MFCC
implementations by comparing against standard libraries (scipy, librosa).

Test Coverage:
    - FFT: correctness, edge cases, performance
    - STFT: correctness, window functions, reconstruction
    - MFCC: correctness, filterbank, delta features

Run:
    pytest tests/test_dsp_core.py -v
    or
    python tests/test_dsp_core.py
"""

import sys
import os
import time

# Add src to path
sys.path.insert(0, os.path.join(os.path.dirname(__file__), '..'))

import numpy as np
from scipy.fft import fft as scipy_fft, rfft as scipy_rfft
from scipy.fftpack import dct as scipy_dct
import librosa

from src.dsp_core import fft, rfft, stft, istft, mfcc, mel_filterbank, delta


class TestFFT:
    """Test suite for FFT implementation."""

    def test_fft_random_signal(self):
        """Test FFT on random signal."""
        x = np.random.randn(1024)
        X_ours = fft(x)
        X_scipy = scipy_fft(x)
        error = np.abs(X_ours - X_scipy)

        print(f"\n[FFT Random Signal]")
        print(f"  Max error: {error.max():.2e}")
        print(f"  Mean error: {error.mean():.2e}")

        assert error.max() < 1e-10, f"FFT error too large: {error.max()}"

    def test_fft_power_of_2(self):
        """Test FFT on power-of-2 lengths."""
        for N in [64, 128, 256, 512, 1024]:
            x = np.random.randn(N)
            X_ours = fft(x)
            X_scipy = scipy_fft(x)
            error = np.abs(X_ours - X_scipy)
            assert error.max() < 1e-10, f"FFT failed for N={N}"

        print(f"\n[FFT Power of 2] All sizes passed ✓")

    def test_fft_non_power_of_2(self):
        """Test FFT on non-power-of-2 lengths."""
        for N in [100, 250, 500, 1000]:
            x = np.random.randn(N)
            X_ours = fft(x)
            X_scipy = scipy_fft(x)
            error = np.abs(X_ours - X_scipy)
            assert error.max() < 1e-10, f"FFT failed for N={N}"

        print(f"\n[FFT Non-Power of 2] All sizes passed ✓")

    def test_fft_sine_wave(self):
        """Test FFT on known sine wave."""
        # Create a pure sine wave at 10 Hz
        sr = 1000
        duration = 1.0
        freq = 10.0
        t = np.arange(int(sr * duration)) / sr
        x = np.sin(2 * np.pi * freq * t)

        X_ours = fft(x)
        X_scipy = scipy_fft(x)

        error = np.abs(X_ours - X_scipy)
        print(f"\n[FFT Sine Wave]")
        print(f"  Frequency: {freq} Hz")
        print(f"  Max error: {error.max():.2e}")

        assert error.max() < 1e-10

    def test_rfft(self):
        """Test real FFT."""
        x = np.random.randn(1024)
        X_ours = rfft(x)
        X_scipy = scipy_rfft(x)

        error = np.abs(X_ours - X_scipy)
        print(f"\n[RFFT]")
        print(f"  Output length: {len(X_ours)} (expected {len(x)//2 + 1})")
        print(f"  Max error: {error.max():.2e}")

        assert len(X_ours) == len(x) // 2 + 1
        assert error.max() < 1e-10

    def test_fft_performance(self):
        """Benchmark FFT performance."""
        sizes = [256, 512, 1024, 2048, 4096]
        print(f"\n[FFT Performance Benchmark]")
        print(f"{'Size':>6s} | {'Ours (ms)':>10s} | {'Scipy (ms)':>11s} | {'Ratio':>6s}")
        print("-" * 50)

        for N in sizes:
            x = np.random.randn(N)

            # Warm up
            _ = fft(x)
            _ = scipy_fft(x)

            # Benchmark ours
            n_iter = 100
            start = time.time()
            for _ in range(n_iter):
                _ = fft(x)
            time_ours = (time.time() - start) / n_iter * 1000

            # Benchmark scipy
            start = time.time()
            for _ in range(n_iter):
                _ = scipy_fft(x)
            time_scipy = (time.time() - start) / n_iter * 1000

            ratio = time_ours / time_scipy
            print(f"{N:6d} | {time_ours:10.3f} | {time_scipy:11.3f} | {ratio:6.2f}x")


class TestSTFT:
    """Test suite for STFT implementation."""

    def test_stft_random_signal(self):
        """Test STFT on random signal."""
        y = np.random.randn(22050)

        D_ours = stft(y, n_fft=2048, hop_length=512, window='hann', center=True)
        D_librosa = librosa.stft(y, n_fft=2048, hop_length=512, window='hann', center=True)

        error = np.abs(D_ours - D_librosa)
        print(f"\n[STFT Random Signal]")
        print(f"  Output shape: {D_ours.shape}")
        print(f"  Max error: {error.max():.2e}")
        print(f"  Mean error: {error.mean():.2e}")

        assert error.max() < 1e-10

    def test_stft_windows(self):
        """Test different window functions."""
        y = np.random.randn(22050)
        windows = ['hann', 'hamming', 'blackman']

        print(f"\n[STFT Window Functions]")
        print(f"{'Window':>10s} | {'Max Error':>12s} | {'Status':>8s}")
        print("-" * 40)

        for win in windows:
            D_ours = stft(y, n_fft=2048, hop_length=512, window=win)
            D_librosa = librosa.stft(y, n_fft=2048, hop_length=512, window=win)

            error = np.abs(D_ours - D_librosa)
            status = "✓ PASS" if error.max() < 1e-10 else "✗ FAIL"
            print(f"{win:>10s} | {error.max():>12.2e} | {status:>8s}")

            assert error.max() < 1e-10

    def test_istft_reconstruction(self):
        """Test STFT/ISTFT perfect reconstruction."""
        y_original = np.random.randn(22050)

        # STFT
        D = stft(y_original, n_fft=2048, hop_length=512, window='hann', center=True)

        # ISTFT - must specify original length for perfect reconstruction
        y_reconstructed = istft(D, hop_length=512, window='hann', center=True, length=len(y_original))

        error = np.abs(y_original - y_reconstructed)
        print(f"\n[ISTFT Reconstruction]")
        print(f"  Max error: {error.max():.2e}")
        print(f"  Mean error: {error.mean():.2e}")

        # Note: reconstruction error is higher due to numerical precision and windowing
        assert error.max() < 1e-6

    def test_stft_performance(self):
        """Benchmark STFT performance."""
        y = np.random.randn(22050)

        print(f"\n[STFT Performance Benchmark]")

        # Warm up
        _ = stft(y, n_fft=2048, hop_length=512)
        _ = librosa.stft(y, n_fft=2048, hop_length=512)

        # Benchmark ours
        n_iter = 50
        start = time.time()
        for _ in range(n_iter):
            _ = stft(y, n_fft=2048, hop_length=512)
        time_ours = (time.time() - start) / n_iter * 1000

        # Benchmark librosa
        start = time.time()
        for _ in range(n_iter):
            _ = librosa.stft(y, n_fft=2048, hop_length=512)
        time_librosa = (time.time() - start) / n_iter * 1000

        ratio = time_ours / time_librosa
        print(f"  Ours:    {time_ours:.2f} ms")
        print(f"  Librosa: {time_librosa:.2f} ms")
        print(f"  Ratio:   {ratio:.2f}x")


class TestMFCC:
    """Test suite for MFCC implementation."""

    def test_mfcc_random_signal(self):
        """Test MFCC on random signal."""
        y = np.random.randn(22050)
        sr = 22050

        mfccs_ours = mfcc(y, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)
        mfccs_librosa = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)

        error = np.abs(mfccs_ours - mfccs_librosa)
        print(f"\n[MFCC Random Signal]")
        print(f"  Output shape: {mfccs_ours.shape}")
        print(f"  Max error: {error.max():.2e}")
        print(f"  Mean error: {error.mean():.2e}")

        # MFCC has slightly higher tolerance due to accumulated numerical errors
        assert error.max() < 1e-4

    def test_mel_filterbank(self):
        """Test Mel filterbank construction."""
        mel_fb_ours = mel_filterbank(sr=22050, n_fft=2048, n_mels=128)
        mel_fb_librosa = librosa.filters.mel(sr=22050, n_fft=2048, n_mels=128)

        error = np.abs(mel_fb_ours - mel_fb_librosa)
        print(f"\n[Mel Filterbank]")
        print(f"  Shape: {mel_fb_ours.shape}")
        print(f"  Max error: {error.max():.2e}")
        print(f"  Mean error: {error.mean():.2e}")
        print(f"  Status: {'✓ PASS' if error.max() < 1e-8 else '✗ FAIL'}")

        # Slightly higher tolerance due to accumulated floating point errors
        assert error.max() < 1e-8

    def test_delta_features(self):
        """Test delta feature computation."""
        y = np.random.randn(22050)
        sr = 22050

        mfccs = mfcc(y, sr=sr, n_mfcc=13)

        delta_ours = delta(mfccs, width=9)
        mfccs_librosa = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        delta_librosa = librosa.feature.delta(mfccs_librosa, width=9)

        error = np.abs(delta_ours - delta_librosa)
        print(f"\n[Delta Features]")
        print(f"  Shape: {delta_ours.shape}")
        print(f"  Max error: {error.max():.2e}")

        assert error.max() < 1e-4

    def test_delta_delta_features(self):
        """Test delta-delta features."""
        y = np.random.randn(22050)
        sr = 22050

        mfccs = mfcc(y, sr=sr, n_mfcc=13)

        delta_delta_ours = delta(mfccs, width=9, order=2)
        mfccs_librosa = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        delta_delta_librosa = librosa.feature.delta(mfccs_librosa, width=9, order=2)

        error = np.abs(delta_delta_ours - delta_delta_librosa)
        print(f"\n[Delta-Delta Features]")
        print(f"  Max error: {error.max():.2e}")

        assert error.max() < 1e-4

    def test_dct(self):
        """Test DCT implementation."""
        from src.dsp_core.mfcc import dct

        x = np.random.randn(128)
        dct_ours = dct(x, norm='ortho')
        dct_scipy = scipy_dct(x, type=2, norm='ortho')

        error = np.abs(dct_ours - dct_scipy)
        print(f"\n[DCT]")
        print(f"  Max error: {error.max():.2e}")

        assert error.max() < 1e-10

    def test_mfcc_performance(self):
        """Benchmark MFCC performance."""
        y = np.random.randn(22050)
        sr = 22050

        print(f"\n[MFCC Performance Benchmark]")

        # Warm up
        _ = mfcc(y, sr=sr, n_mfcc=13)
        _ = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        # Benchmark ours
        n_iter = 20
        start = time.time()
        for _ in range(n_iter):
            _ = mfcc(y, sr=sr, n_mfcc=13)
        time_ours = (time.time() - start) / n_iter * 1000

        # Benchmark librosa
        start = time.time()
        for _ in range(n_iter):
            _ = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        time_librosa = (time.time() - start) / n_iter * 1000

        ratio = time_ours / time_librosa
        print(f"  Ours:    {time_ours:.2f} ms")
        print(f"  Librosa: {time_librosa:.2f} ms")
        print(f"  Ratio:   {ratio:.2f}x")


def run_all_tests():
    """Run all test suites."""
    print("=" * 70)
    print("DSP Core Module - Comprehensive Unit Tests")
    print("=" * 70)

    # FFT Tests
    print("\n" + "=" * 70)
    print("FFT Tests")
    print("=" * 70)
    test_fft = TestFFT()
    test_fft.test_fft_random_signal()
    test_fft.test_fft_power_of_2()
    test_fft.test_fft_non_power_of_2()
    test_fft.test_fft_sine_wave()
    test_fft.test_rfft()
    test_fft.test_fft_performance()

    # STFT Tests
    print("\n" + "=" * 70)
    print("STFT Tests")
    print("=" * 70)
    test_stft = TestSTFT()
    test_stft.test_stft_random_signal()
    test_stft.test_stft_windows()
    test_stft.test_istft_reconstruction()
    test_stft.test_stft_performance()

    # MFCC Tests
    print("\n" + "=" * 70)
    print("MFCC Tests")
    print("=" * 70)
    test_mfcc = TestMFCC()
    test_mfcc.test_mfcc_random_signal()
    test_mfcc.test_mel_filterbank()
    test_mfcc.test_delta_features()
    test_mfcc.test_delta_delta_features()
    test_mfcc.test_dct()
    test_mfcc.test_mfcc_performance()

    print("\n" + "=" * 70)
    print("ALL TESTS PASSED ✓")
    print("=" * 70)


if __name__ == "__main__":
    run_all_tests()


# DSP Algorithm Verification

This notebook verifies the correctness of our hand-written FFT, STFT, and MFCC implementations by comparing against standard libraries (scipy, librosa).

**Contents:**
1. FFT Verification
2. STFT Verification
3. MFCC Verification
4. Performance Benchmarks
5. Visualizations

**Author:** DSP-HW Team  
**Date:** 2025-12-07

In [ ]:
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft as scipy_fft
from scipy.signal import get_window as scipy_window
import librosa
import librosa.display

# Add src to path
sys.path.insert(0, os.path.abspath('..'))

from src.dsp_core import fft, rfft, stft, istft, mfcc, mel_filterbank, delta

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ All imports successful")

## 1. FFT Verification

We test FFT on various signals and compare with scipy.fft.

In [ ]:
# Test 1: Sine wave (known ground truth)
sr = 1000  # Sample rate
duration = 1.0
freq = 10.0  # 10 Hz sine wave
t = np.arange(int(sr * duration)) / sr
x = np.sin(2 * np.pi * freq * t)

# Compute FFT
X_ours = fft(x)
X_scipy = scipy_fft(x)

# Compute error
error = np.abs(X_ours - X_scipy)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Time domain
axes[0, 0].plot(t[:100], x[:100])
axes[0, 0].set_title('Input Signal (10 Hz Sine Wave)')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].grid(True)

# Magnitude spectrum (ours)
freqs = np.fft.fftfreq(len(x), 1/sr)
axes[0, 1].plot(freqs[:len(freqs)//2], np.abs(X_ours)[:len(freqs)//2])
axes[0, 1].set_title('FFT Magnitude (Our Implementation)')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_ylabel('Magnitude')
axes[0, 1].set_xlim([0, 50])
axes[0, 1].grid(True)

# Magnitude spectrum (scipy)
axes[1, 0].plot(freqs[:len(freqs)//2], np.abs(X_scipy)[:len(freqs)//2])
axes[1, 0].set_title('FFT Magnitude (SciPy Reference)')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('Magnitude')
axes[1, 0].set_xlim([0, 50])
axes[1, 0].grid(True)

# Error plot
axes[1, 1].semilogy(error)
axes[1, 1].set_title(f'Absolute Error (Max: {error.max():.2e})')
axes[1, 1].set_xlabel('Frequency Bin')
axes[1, 1].set_ylabel('Error')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"FFT Verification:")
print(f"  Max error: {error.max():.2e}")
print(f"  Mean error: {error.mean():.2e}")
print(f"  Status: {'✓ PASS' if error.max() < 1e-10 else '✗ FAIL'}")

In [ ]:
# Test 2: Random signal
x_random = np.random.randn(2048)
X_ours_random = fft(x_random)
X_scipy_random = scipy_fft(x_random)
error_random = np.abs(X_ours_random - X_scipy_random)

print(f"\nFFT on Random Signal:")
print(f"  Length: {len(x_random)}")
print(f"  Max error: {error_random.max():.2e}")
print(f"  Mean error: {error_random.mean():.2e}")
print(f"  Status: {'✓ PASS' if error_random.max() < 1e-10 else '✗ FAIL'}")

## 2. STFT Verification

We test STFT on audio signals and compare with librosa.stft.

In [ ]:
# Generate test signal: chirp (frequency sweep)
sr = 22050
duration = 2.0
t = np.arange(int(sr * duration)) / sr
# Chirp from 100 Hz to 2000 Hz
f0, f1 = 100, 2000
y = np.sin(2 * np.pi * (f0 + (f1 - f0) * t / duration) * t)

# Compute STFT
n_fft = 2048
hop_length = 512

D_ours = stft(y, n_fft=n_fft, hop_length=hop_length, window='hann', center=True)
D_librosa = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window='hann', center=True)

# Compute error
error_stft = np.abs(D_ours - D_librosa)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time domain
axes[0, 0].plot(t, y)
axes[0, 0].set_title('Input Signal (Chirp: 100-2000 Hz)')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].grid(True)

# Spectrogram (ours)
librosa.display.specshow(
    librosa.amplitude_to_db(np.abs(D_ours), ref=np.max),
    sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz',
    ax=axes[0, 1], cmap='viridis'
)
axes[0, 1].set_title('STFT Spectrogram (Our Implementation)')
axes[0, 1].set_ylim([0, 3000])

# Spectrogram (librosa)
librosa.display.specshow(
    librosa.amplitude_to_db(np.abs(D_librosa), ref=np.max),
    sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz',
    ax=axes[1, 0], cmap='viridis'
)
axes[1, 0].set_title('STFT Spectrogram (Librosa Reference)')
axes[1, 0].set_ylim([0, 3000])

# Error heatmap
im = axes[1, 1].imshow(
    np.log10(error_stft + 1e-15),
    aspect='auto', origin='lower', cmap='hot',
    extent=[0, duration, 0, sr/2000]
)
axes[1, 1].set_title(f'Error (log scale, Max: {error_stft.max():.2e})')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Frequency (kHz)')
plt.colorbar(im, ax=axes[1, 1], label='log10(error)')

plt.tight_layout()
plt.show()

print(f"\nSTFT Verification:")
print(f"  Output shape: {D_ours.shape}")
print(f"  Max error: {error_stft.max():.2e}")
print(f"  Mean error: {error_stft.mean():.2e}")
print(f"  Status: {'✓ PASS' if error_stft.max() < 1e-10 else '✗ FAIL'}")

In [ ]:
# Test ISTFT reconstruction
y_reconstructed = istft(D_ours, hop_length=hop_length, window='hann', center=True, length=len(y))
reconstruction_error = np.abs(y - y_reconstructed)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Original vs reconstructed
axes[0].plot(t[:1000], y[:1000], label='Original', alpha=0.7)
axes[0].plot(t[:1000], y_reconstructed[:1000], label='Reconstructed', alpha=0.7, linestyle='--')
axes[0].set_title('STFT/ISTFT Reconstruction')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].legend()
axes[0].grid(True)

# Reconstruction error
axes[1].plot(t, reconstruction_error)
axes[1].set_title(f'Reconstruction Error (Max: {reconstruction_error.max():.2e})')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Error')
axes[1].grid(True)

plt.tight_layout()
plt.show()

print(f"\nISTFT Reconstruction:")
print(f"  Max error: {reconstruction_error.max():.2e}")
print(f"  Mean error: {reconstruction_error.mean():.2e}")
print(f"  Status: {'✓ PASS' if reconstruction_error.max() < 1e-6 else '✗ FAIL'}")

## 3. MFCC Verification

We test MFCC feature extraction and compare with librosa.feature.mfcc.

In [ ]:
# Load a real audio file from ESC-50
# If not available, use synthetic signal
audio_path = '../ESC-50/audio/1-100032-A-0.wav'

if os.path.exists(audio_path):
    y_audio, sr = librosa.load(audio_path, sr=22050)
    print(f"Loaded audio: {audio_path}")
else:
    # Use synthetic chirp
    y_audio = y
    sr = 22050
    print("Using synthetic chirp signal")

# Compute MFCC
mfccs_ours = mfcc(y_audio, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)
mfccs_librosa = librosa.feature.mfcc(y=y_audio, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)

# Compute error
error_mfcc = np.abs(mfccs_ours - mfccs_librosa)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MFCC (ours)
img1 = librosa.display.specshow(
    mfccs_ours, sr=sr, hop_length=512,
    x_axis='time', ax=axes[0, 0], cmap='coolwarm'
)
axes[0, 0].set_title('MFCC (Our Implementation)')
axes[0, 0].set_ylabel('MFCC Coefficient')
plt.colorbar(img1, ax=axes[0, 0])

# MFCC (librosa)
img2 = librosa.display.specshow(
    mfccs_librosa, sr=sr, hop_length=512,
    x_axis='time', ax=axes[0, 1], cmap='coolwarm'
)
axes[0, 1].set_title('MFCC (Librosa Reference)')
axes[0, 1].set_ylabel('MFCC Coefficient')
plt.colorbar(img2, ax=axes[0, 1])

# Error heatmap
img3 = librosa.display.specshow(
    error_mfcc, sr=sr, hop_length=512,
    x_axis='time', ax=axes[1, 0], cmap='hot'
)
axes[1, 0].set_title(f'Absolute Error (Max: {error_mfcc.max():.2e})')
axes[1, 0].set_ylabel('MFCC Coefficient')
plt.colorbar(img3, ax=axes[1, 0])

# Error distribution
axes[1, 1].hist(error_mfcc.flatten(), bins=50, edgecolor='black')
axes[1, 1].set_title('Error Distribution')
axes[1, 1].set_xlabel('Error')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"\nMFCC Verification:")
print(f"  Output shape: {mfccs_ours.shape}")
print(f"  Max error: {error_mfcc.max():.2e}")
print(f"  Mean error: {error_mfcc.mean():.2e}")
print(f"  Median error: {np.median(error_mfcc):.2e}")
print(f"  Status: {'✓ PASS' if error_mfcc.max() < 1e-4 else '✗ FAIL'}")

In [ ]:
# Test Mel filterbank
mel_fb_ours = mel_filterbank(sr=22050, n_fft=2048, n_mels=128)
mel_fb_librosa = librosa.filters.mel(sr=22050, n_fft=2048, n_mels=128)

error_mel = np.abs(mel_fb_ours - mel_fb_librosa)

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Our filterbank
im1 = axes[0].imshow(mel_fb_ours, aspect='auto', origin='lower', cmap='viridis')
axes[0].set_title('Mel Filterbank (Our Implementation)')
axes[0].set_xlabel('Frequency Bin')
axes[0].set_ylabel('Mel Band')
plt.colorbar(im1, ax=axes[0])

# Librosa filterbank
im2 = axes[1].imshow(mel_fb_librosa, aspect='auto', origin='lower', cmap='viridis')
axes[1].set_title('Mel Filterbank (Librosa Reference)')
axes[1].set_xlabel('Frequency Bin')
axes[1].set_ylabel('Mel Band')
plt.colorbar(im2, ax=axes[1])

# Error
im3 = axes[2].imshow(error_mel, aspect='auto', origin='lower', cmap='hot')
axes[2].set_title(f'Error (Max: {error_mel.max():.2e})')
axes[2].set_xlabel('Frequency Bin')
axes[2].set_ylabel('Mel Band')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

print(f"\nMel Filterbank Verification:")
print(f"  Shape: {mel_fb_ours.shape}")
print(f"  Max error: {error_mel.max():.2e}")
print(f"  Mean error: {error_mel.mean():.2e}")
print(f"  Status: {'✓ PASS' if error_mel.max() < 1e-8 else '✗ FAIL'}")

## 4. Performance Benchmarks

We compare execution time of our implementations vs standard libraries.

In [ ]:
import pandas as pd

# FFT performance
sizes = [256, 512, 1024, 2048, 4096]
results = []

for N in sizes:
    x = np.random.randn(N)
    
    # Warm up
    _ = fft(x)
    _ = scipy_fft(x)
    
    # Benchmark
    n_iter = 100
    
    start = time.time()
    for _ in range(n_iter):
        _ = fft(x)
    time_ours = (time.time() - start) / n_iter * 1000
    
    start = time.time()
    for _ in range(n_iter):
        _ = scipy_fft(x)
    time_scipy = (time.time() - start) / n_iter * 1000
    
    results.append({
        'Size': N,
        'Ours (ms)': time_ours,
        'SciPy (ms)': time_scipy,
        'Ratio': time_ours / time_scipy
    })

df_fft = pd.DataFrame(results)
print("\nFFT Performance Benchmark:")
print(df_fft.to_string(index=False))

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_fft['Size'], df_fft['Ours (ms)'], marker='o', label='Our Implementation')
ax.plot(df_fft['Size'], df_fft['SciPy (ms)'], marker='s', label='SciPy')
ax.set_xlabel('FFT Size')
ax.set_ylabel('Time (ms)')
ax.set_title('FFT Performance Comparison')
ax.legend()
ax.grid(True)
ax.set_xscale('log', base=2)
ax.set_yscale('log')
plt.tight_layout()
plt.show()

## 5. Summary

Final verification summary with all metrics.

In [ ]:
# Compile all results
summary = pd.DataFrame([
    {
        'Algorithm': 'FFT (Sine Wave)',
        'Max Error': f"{error.max():.2e}",
        'Mean Error': f"{error.mean():.2e}",
        'Status': '✓ PASS' if error.max() < 1e-10 else '✗ FAIL'
    },
    {
        'Algorithm': 'FFT (Random)',
        'Max Error': f"{error_random.max():.2e}",
        'Mean Error': f"{error_random.mean():.2e}",
        'Status': '✓ PASS' if error_random.max() < 1e-10 else '✗ FAIL'
    },
    {
        'Algorithm': 'STFT',
        'Max Error': f"{error_stft.max():.2e}",
        'Mean Error': f"{error_stft.mean():.2e}",
        'Status': '✓ PASS' if error_stft.max() < 1e-10 else '✗ FAIL'
    },
    {
        'Algorithm': 'ISTFT',
        'Max Error': f"{reconstruction_error.max():.2e}",
        'Mean Error': f"{reconstruction_error.mean():.2e}",
        'Status': '✓ PASS' if reconstruction_error.max() < 1e-6 else '✗ FAIL'
    },
    {
        'Algorithm': 'MFCC',
        'Max Error': f"{error_mfcc.max():.2e}",
        'Mean Error': f"{error_mfcc.mean():.2e}",
        'Status': '✓ PASS' if error_mfcc.max() < 1e-6 else '✗ FAIL'
    },
    {
        'Algorithm': 'Mel Filterbank',
        'Max Error': f"{error_mel.max():.2e}",
        'Mean Error': f"{error_mel.mean():.2e}",
        'Status': '✓ PASS' if error_mel.max() < 1e-8 else '✗ FAIL'
    },
])

print("\n" + "="*70)
print("VERIFICATION SUMMARY")
print("="*70)
print(summary.to_string(index=False))
print("="*70)

# Check if all passed
all_passed = all('✓' in status for status in summary['Status'])
if all_passed:
    print("\n🎉 ALL ALGORITHMS VERIFIED SUCCESSFULLY! 🎉")
else:
    print("\n⚠️  Some algorithms failed verification. Please review.")